
# 如果你需要挂载GoogleDrive，请点亮代码块右侧的Mounting_Google_Drive选项。
理论上代码块可以直接运行且只需运行一次，如果你遇到程序意外中断的情况请在Github的Issue区进行反馈。


---


代码执行执行时间预计为20分钟，请做好心理准备，但如果你已经在GoogleDrive中保存好了项目文件和模型，那么代码执行时间可能会缩短至10分钟。


---


如果你还没在GoogleDrive中保存项目文件但依然点亮了Mounting_Google_Drive选项，那么程序会自动在你的Drive中保存项目所需的所有文件和模型，以便于以后读取，模型和文件的总体积大概在5G左右，清注意管理存储空间。

In [ ]:
Mounting_Google_Drive = False  #@param {type:"boolean"}

if Mounting_Google_Drive:
    # 挂载Google Drive
    from google.colab import drive
    import os

    drive.mount('/content/drive')

    # 检测 GPT-SoVITS 文件夹是否存在
    gpt_sovits_path = "/content/drive/MyDrive/GPT-SoVITS"
    if not os.path.exists(gpt_sovits_path):
        # 如果文件夹不存在，则克隆
        !pip install -q condacolab
        import condacolab
        condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py39_23.11.0-2-Linux-x86_64.sh")
        %cd -q /content/drive/MyDrive
        !git clone https://github.com/RVC-Boss/GPT-SoVITS
        gpt_sovits_exists = False
    else:
        # 如果文件夹存在，则更新
        %cd -q /content/drive/MyDrive/GPT-SoVITS
        !git pull origin main
        gpt_sovits_exists = True

    %cd -q /content/drive/MyDrive/GPT-SoVITS

    # 准备运行环境
    !conda create -n GPTSoVits python=3.9
    !conda init
    !conda activate GPTSoVits
    !conda install ipykernel
    !bash install.sh


    import shutil

    pretrained_models_path = "/content/drive/MyDrive/GPT-SoVITS/GPT_SoVITS/pretrained_models"
    if not os.path.exists(pretrained_models_path + "/GPT-SoVITS"):
        %cd "{pretrained_models_path}"
        !git clone https://huggingface.co/lj1995/GPT-SoVITS
    else:
        %cd "{pretrained_models_path}/GPT-SoVITS"
        !git pull origin main

    # 这行代码会导致模型占用更多空间，但可以避免一个棘手的问题
    if len(os.listdir(pretrained_models_path + "/GPT-SoVITS")) > 0:
        !cp -r "/content/drive/MyDrive/GPT-SoVITS/GPT_SoVITS/pretrained_models/GPT-SoVITS/." "/content/drive/MyDrive/GPT-SoVITS/GPT_SoVITS/pretrained_models/"

    # 创建ASR模型目录
    asr_models_path = "/content/drive/MyDrive/GPT-SoVITS/tools/damo_asr/models"
    if not os.path.exists(asr_models_path):
        os.makedirs(asr_models_path)
        %cd "{asr_models_path}"
        !git clone https://www.modelscope.cn/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch.git
        !git clone https://www.modelscope.cn/damo/speech_fsmn_vad_zh-cn-16k-common-pytorch.git
        !git clone https://www.modelscope.cn/damo/punc_ct-transformer_zh-cn-common-vocab272727-pytorch.git
    else:
        # 这部分代码不起作用，但不影响功能，暂时保留
        %cd "{asr_models_path}/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch"
        !git pull origin main

        %cd "{asr_models_path}/speech_fsmn_vad_zh-cn-16k-common-pytorch"
        !git pull origin main

        %cd "{asr_models_path}/punc_ct-transformer_zh-cn-common-vocab272727-pytorch"
        !git pull origin main

    # 创建UV5模型目录
    uvr5_models_path = "/content/drive/MyDrive/GPT-SoVITS/tools/uvr5"
    if not os.path.exists(uvr5_models_path + "/uvr5_weights"):
        os.makedirs(uvr5_models_path)
        %cd "{uvr5_models_path}"
        !git clone https://huggingface.co/Delik/uvr5_weights
        !git config core.sparseCheckout true
    else:
        %cd "{uvr5_models_path}/uvr5_weights"
        !git pull origin main

    # 修改配置文件中的值
    !sed -i '10s/False/True/' "/content/drive/MyDrive/GPT-SoVITS/config.py"

    # 进入 GPT-SoVITS 目录并启动 WebUI
    %cd "/content/drive/MyDrive/GPT-SoVITS/"
    !/usr/local/bin/python webui.py

else:

    # 以下为初版colab notebook的代码，唯一的区别是原先有关运行环境的代码会导致runtime重启，现在该部分的代码已被修改。
    !pip install -q condacolab
    # Setting up condacolab and installing packages
    import condacolab
    condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py39_23.11.0-2-Linux-x86_64.sh")
    %cd -q /content
    !git clone https://github.com/RVC-Boss/GPT-SoVITS

    %cd -q /content/GPT-SoVITS

    !conda create -n GPTSoVits python=3.9
    !conda init
    !conda activate GPTSoVits
    !conda install ipykernel
    !bash install.sh

    # @title Download pretrained models 下载预训练模型
    !mkdir -p /content/GPT-SoVITS/GPT_SoVITS/pretrained_models
    !mkdir -p /content/GPT-SoVITS/tools/damo_asr/models
    !mkdir -p /content/GPT-SoVITS/tools/uvr5
    %cd /content/GPT-SoVITS/GPT_SoVITS/pretrained_models
    !git clone https://huggingface.co/lj1995/GPT-SoVITS
    %cd /content/GPT-SoVITS/tools/damo_asr/models
    !git clone https://www.modelscope.cn/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch.git
    !git clone https://www.modelscope.cn/damo/speech_fsmn_vad_zh-cn-16k-common-pytorch.git
    !git clone https://www.modelscope.cn/damo/punc_ct-transformer_zh-cn-common-vocab272727-pytorch.git
    # @title UVR5 pretrains 安装uvr5模型
    %cd /content/GPT-SoVITS/tools/uvr5
    !git clone https://huggingface.co/Delik/uvr5_weights
    !git config core.sparseCheckout true
    !mv /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/GPT-SoVITS/* /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/

    # 启动UI
    !sed -i '10s/False/True/' /content/GPT-SoVITS/config.py
    %cd /content/GPT-SoVITS/
    !/usr/local/bin/python  webui.py